In [2]:
!pip --q install openai langchain langchain-community langchain-core
!pip --q install pytube youtube-transcript-api tiktoken

In [3]:
import os

os.environ["OPENAI_API_KEY"] = '45462e152b294e11bbcd89f632b6a4df'
os.environ["OPENAI_API_TYPE"] = 'azure'
os.environ["OPENAI_API_VERSION"] = '2023-05-15'
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://sktflyailab.openai.azure.com/'

In [5]:
from langchain.document_loaders import YoutubeLoader
loader = YoutubeLoader.from_youtube_url('https://youtu.be/c_Re0fcMeFg?si=yHwCSCxyehXIQu_m',
                                        language=['ko'],
                                        add_video_info=True)

docs = loader.load()
docs

[Document(metadata={'source': 'c_Re0fcMeFg', 'title': '티라노사우루스가 유일하게 이기지 못했던 공룡', 'description': 'Unknown', 'view_count': 1845, 'thumbnail_url': 'https://i.ytimg.com/vi/c_Re0fcMeFg/hq720.jpg', 'publish_date': '2024-08-07 00:00:00', 'length': 486, 'author': '탐사튜브'}, page_content='지구에서 가장 강력했던 육상 포식자 수각류 육식공룡 버스 크기까지 자라던 이들은 분명 생태계의 최상이 포식자였다 그렇듯 녀석들도 당시 생태계에서 가장 강력한 공룡은 아니었죠 동물의 왕이라 불리는 사자보다 코끼리가 훨씬 크고 강력하이 공룡 시대에 진정한 왕을 꼽는다면 건물처럼 거대한 용각류 왕좌를 차지할 겁니다 길이가 몇십 미터를 훌쩍 넘고 체중도 수십톤의 달하던 이들은 그 당시 신과 같은 존재였습니다 그리고이는 중생대의 마지막인 6,600만 년전 북아메리카 대륙에 살았던 한 용각류 게도 해당하는 말이었죠 거대한 티타노사우루스 루스는 당시 생태계에서 가장 크고 강력한 동물이었습니다 성체는 보통 몸길이 20에서 26m 체중은 30에서 35톤 어깨만 5m 달할만큼 거대했습니다 심지어 발견된 가장 큰 개체는 체중 38톤 달할 정도였죠 하지만이 거신에도 눈의 가시가 있었으니 바로 공룡의 왕이라 불리는 티라노사우루스 있습니다 지구에 존재했던 가장 강력한 수강 류인 녀석은 60 공룡 중 유일하게 10톤을 넘게 자할 수 있었다 알려져 있는데요 알라모사우루스 비하면 작았지만 호식자 치고는 상당한 덩치를 자랑했습니다 최근에는이 괴물 같은 수각류 최대 의 크기가 무려 15톤의 달할 수 있었다는 논문이 발표되기도 했죠 물론 이런 극단적인 예시를 제외하더라도 녀석은 평균적으로 8톤의 넘던 포식자였다 아우루스 이런 크기는 알라모사우루스게 있어도 결코 강구할 수 없는 것이었죠 이렇듯 같은시기 같

In [6]:
# 유튜브 tts 성능이 상당히 좋다 => 위에 유튜브 내용을 gpt한테 요약시킬 수 있다.
from langchain.prompts import PromptTemplate
from langchain.chains import load_summarize_chain


In [ ]:
# 내용을 한국어로 요약하는 함수
def kr_summurize(docs):
  map_template ='''
  다음은 유튜브 영상 자막의 일부야. 주요 내용을 한글 200자 내외로 요약해줘.
  -자막-
  {text}
  '''
  map_prompt = PromptTemplate(template=map_template, input_variables=['text'])

In [7]:
# chat모델 만들기
from langchain.chat_models import AzureChatOpenAI

chatgpt = AzureChatOpenAI(
    deployment_name="dev-gpt-4-32k",
    max_tokens=2000,
    temperature=0
)

In [8]:
from langchain.text_splitter import TokenTextSplitter # 토큰 단위로 텍스트 분리

text_splitter = TokenTextSplitter(chunk_size=2000, chunk_overlap=50)
docs = text_splitter.split_documents(docs)

In [9]:
len(docs) #4 묶음 즉 8000토큰의 데이터가 2000 토큰단위로 분리됨

4

In [10]:
#Chain 생성
sum_chain = load_summarize_chain(
    llm=chatgpt,
    chain_type='map_reduce',
    return_intermediate_steps=True,
    verbose=True
)

In [13]:
sum_chain({'input_documents':docs},return_only_outputs=True)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"지구에서 가장 강력했던 육상 포식자 수각류 육식공룡 버스 크기까지 자라던 이들은 분명 생태계의 최상이 포식자였다 그렇듯 녀석들도 당시 생태계에서 가장 강력한 공룡은 아니었죠 동물의 왕이라 불리는 사자보다 코끼리가 훨씬 크고 강력하이 공룡 시대에 진정한 왕을 꼽는다면 건물처럼 거대한 용각류 왕좌를 차지할 겁니다 길이가 몇십 미터를 훌쩍 넘고 체중도 수십톤의 달하던 이들은 그 당시 신과 같은 존재였습니다 그리고이는 중생대의 마지막인 6,600만 년전 북아메리카 대륙에 살았던 한 용각류 게도 해당하는 말이었죠 거대한 티타노사우루스 루스는 당시 생태계에서 가장 크고 강력한 동물이었습니다 성체는 보통 몸길이 20에서 26m 체중은 30에서 35톤 어깨만 5m 달할만큼 거대했습니다 심지어 발견된 가장 큰 개체는 체중 38톤 달할 정도였죠 하지만이 거신에도 눈의 가시가 있었으니 바로 공룡의 왕이라 불리는 티라노사우루스 있습니다 지구에 존재했던 가장 강력한 수강 류인 녀석은 60 공룡 중 유일하게 10톤을 넘게 자할 수 있었다 알려져 있는데요 알라모사우루스 비하면 작았지만 호식자 치고는 상당한 덩치를 자랑했습니다 최근에는이 괴물 같은 수각류 최대 의 크기가 무려 15톤의 달할 수 있었다는 논문이 발표되기도 했죠 물론 이런 극단적인 예시를 제외하더라도 녀석은 평균적으로 8톤의 넘던 포식자였다 아우루스 이런 크기는 알라모사우루스게 있어도 결코 강구할 수 없는 것이었죠 이렇듯 같은시기 같은 지역에 두 괴물 같은 녀석이 공존했다 보니 사람들은 두 동물의 충돌에 대해 추측하기 시작했습니다 티라노사우루스가 무리를 지어 알라모사우루스 사냥했을 것이란 추측 집채만한 알라모사우루스게 티라노사우루스는 아무런 위협이 되지

{'intermediate_steps': ['The article discusses the two most powerful dinosaurs that lived on Earth, the Alamosaurus and the Tyrannosaurus. The Alamosaurus, a herbivore, was the largest and most powerful animal in its ecosystem, reaching lengths of 20 to 26 meters and weights of 30 to 35 tons. The Tyrannosaurus, a carnivore, was smaller but still formidable, being the only known dinosaur capable of growing over 10 tons. Despite their size difference, both dinosaurs coexisted in the same period and region, leading to speculation about potential encounters between the two. The discovery of a fossil named SSM 542 8 has provided crucial evidence in this discussion.',
  "The fossil named SSM 542 8, a well-preserved Alamosaurus specimen, showed signs of being scratched by a sharp object. Scholars believe this was caused by a large Tyrannosaurus, the only animal at the time capable of leaving such marks. This suggests that the Tyrannosaurus and Alamosaurus interacted, with at least one cleanin